In [1]:
!pip install pyspark
import pyspark

from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=25d75600c7636c8a457fc51a81a7a9a84eb62adf66300e4253522842f2881913
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").appName('chapter_9').getOrCreate()

In [3]:
!curl -O ftp://ftp.ncbi.nlm.nih.gov/1000genomes/ftp/phase3/data\
/HG00103/alignment/HG00103.mapped.ILLUMINA.bwa.GBR\
.low_coverage.20120522.bam

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (78) The file does not exist
curl: (3) URL using bad/illegal format or missing URL
curl: (6) Could not resolve host: .low_coverage.20120522.bam


In [4]:
!mv HG00103.mapped.ILLUMINA.bwa.GBR\
.low_coverage.20120522.bam data/genomics

mv: target 'data/genomics' is not a directory


In [ ]:
!adam-submit \
  --master yarn \ 1
  --deploy-mode client \
  --driver-memory 8G \
  --num-executors 6 \
  --executor-cores 4 \
  --executor-memory 12G \
  -- \
  transform \ 2
  data/genomics/HG00103.mapped.ILLUMINA.bwa.GBR\
.low_coverage.20120522.bam \
  data/genomics/HG00103

In [ ]:
!du -sh data/genomics/HG00103*bam
16G  data/genomics/HG00103. [...] .bam

!du -sh data/genomics/HG00103/
13G  data/genomics/HG00103

In [ ]:
!pyadam

In [ ]:
!pyspark --conf spark.serializer=org.apache.spark.
serializer.KryoSerializer --conf spark.kryo.registrator=
org.bdgenomics.adam.serialization.ADAMKryoRegistrator
--jars `find-adam-assembly.sh` --driver-class-path
`find-adam-assembly.sh`

In [ ]:
from bdgenomics.adam.adamContext import ADAMContext

ac = ADAMContext(spark)

readsData = ac.loadAlignments("data/HG00103")

readsDataDF = readsData.toDF()
readsDataDF.show(1, vertical=True)

In [ ]:
readsData.toDF().count()

In [ ]:
unique_chr = readsDataDF.select('referenceName').distinct().collect()
unique_chr = [u.referenceName for u in unique_chr]

unique_chr.sort()

In [ ]:
from pyspark.sql import functions as fun

cftr_reads = readsDataDF.where("referenceName == 7").\
              where(fun.col("start") <= 117149189).\
              where(fun.col("end") > 117149189)

cftr_reads.count()

In [ ]:
!mkdir data/genomics/dnase

In [ ]:
!curl -O -L "https://www.encodeproject.org/ \
              files/ENCFF001UVC/@@download/ENCFF001UVC.bed.gz" | \
              gunzip > data/genomics/dnase/GM12878.DNase.narrowPeak 1
!curl -O -L "https://www.encodeproject.org/ \
              files/ENCFF001UWQ/@@download/ENCFF001UWQ.bed.gz" | \
              gunzip > data/genomics/dnase/K562.DNase.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
              files/ENCFF001WEI/@@download/ENCFF001WEI.bed.gz" | \
              gunzip > data/genomics/dnase/BJ.DNase.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
              files/ENCFF001UVQ/@@download/ENCFF001UVQ.bed.gz" | \
              gunzip > data/genomics/dnase/HEK293.DNase.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001SOM/@@download/ENCFF001SOM.bed.gz" | \
            gunzip > data/genomics/dnase/H54.DNase.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001UVU/@@download/ENCFF001UVU.bed.gz" | \
            gunzip > data/genomics/dnase/HepG2.DNase.narrowPeak

In [ ]:
!mkdir data/genomics/chip-seq

In [ ]:
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001VED/@@download/ENCFF001VED.bed.gz" | \
            gunzip > data/genomics/chip-seq/GM12878.ChIP-seq.CTCF.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001VMZ/@@download/ENCFF001VMZ.bed.gz" | \
            gunzip > data/genomics/chip-seq/K562.ChIP-seq.CTCF.narrowPeak
!!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001XMU/@@download/ENCFF001XMU.bed.gz" | \
            gunzip > data/genomics/chip-seq/BJ.ChIP-seq.CTCF.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001XQU/@@download/ENCFF001XQU.bed.gz" | \
            gunzip > data/genomics/chip-seq/HEK293.ChIP-seq.CTCF.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001USC/@@download/ENCFF001USC.bed.gz" | \
            gunzip> data/genomics/chip-seq/H54.ChIP-seq.CTCF.narrowPeak
!curl -O -L "https://www.encodeproject.org/ \
            files/ENCFF001XRC/@@download/ENCFF001XRC.bed.gz" | \
            gunzip> data/genomics/chip-seq/HepG2.ChIP-seq.CTCF.narrowPeak

!curl -s -L "http://ftp.ebi.ac.uk/pub/databases/gencode/\
            Gencode_human/release_18/gencode.v18.annotation.gtf.gz" | \
            gunzip > data/genomics/gencode.v18.annotation.gtf

In [ ]:
cell_lines = ["GM12878", "K562", "BJ", "HEK293", "H54", "HepG2"]
#for cell in cell_lines:
## For each cell line…
  ## …generate a suitable DataFrame
## Concatenate the DataFrames and carry through into MLlib, for example

In [ ]:
ocal_prefix = "data/genomics"
import pyspark.sql.functions as fun

## UDF for finding closest transcription start site
## naive; exercise for reader: make this faster
def distance_to_closest(loci, query):
  return min([abs(x - query) for x in loci])
distance_to_closest_udf = fun.udf(distance_to_closest)


## build in-memory structure for computing distance to TSS
## we are essentially implementing a broadcast join here
tss_data = ac.loadFeatures("data/genomics/gencode.v18.annotation.gtf")
tss_df = tss_data.toDF().filter(fun.col("featureType") == 'transcript')
b_tss_df = spark.sparkContext.broadcast(tss_df.groupBy('referenceName').\
                agg(fun.collect_list("start").alias("start_sites")))

In [ ]:
current_cell_line = cell_lines[0]

dnase_path = f'data/genomics/dnase/{current_cell_line}.DNase.narrowPeak'
dnase_data = ac.loadFeatures(dnase_path)
dnase_data.toDF().columns

In [ ]:
chip_seq_path = f'data/genomics/chip-seq/ \
                  {current_cell_line}.ChIP-seq.CTCF.narrowPeak'
chipseq_data = ac.loadFeatures(chipseq_path)

In [ ]:
dnase_with_label = dnase_data.leftOuterShuffleRegionJoin(chipseq_data)
dnase_with_label_df = dnase_with_label.toDF()

In [ ]:
dnase_with_label_df = dnase_with_label_df.\
                        withColumn("label", \
                                    ~fun.col("_2").isNull())
dnase_with_label_df.show(5)

In [ ]:
## build final training DF
training_df = dnase_with_label_df.withColumn(
    "contig", fun.col("_1").referenceName).withColumn(
    "start", fun.col("_1").start).withColumn(
    "end", fun.col("_1").end).withColumn(
    "tf", fun.lit("CTCF")).withColumn(
    "cell_line", fun.lit(current_cell_line)).drop("_1", "_2")

training_df = training_df.join(b_tss_df,
                               training_df.contig == b_tss_df.referenceName,
                               "inner")

training_df.withColumn("closest_tss",
                      fun.least(distance_to_closest_udf(fun.col("start_sites"),
                                                        fun.col("start")),
                          distance_to_closest_udf(fun.col("start_sites"),
                                                  fun.col("end"))))

In [ ]:
preTrainingData = data_by_cellLine.union(...)
preTrainingData.cache()

preTrainingData.count()
preTrainingData.filter(fun.col("label") == true).count()